In [1]:
#!pip install torch>=2.1.0 torchvision torchaudio

In [2]:
from datasets import load_dataset
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

/home/ec2-user/software/miniconda3/envs/week2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#1 data
dataset=load_dataset("imdb")
train_dataset=dataset["train"].shuffle(seed=42).select(range(4000))
test_dataset=dataset["test"].shuffle(seed=42).select(range(4000))
print(train_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 4000
})


In [4]:
#2 处理导入
#token输入序列
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_function(example):
    return tokenizer(example["text"],padding="max_length",truncation=True) #如何对句子进行token化 填充到指定长度，trunca截断

train_dataset=train_dataset.map(tokenize_function,batched=True)
test_dataset=test_dataset.map(tokenize_function,batched=True)
#print(train_dataset[1])
"""
#['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask']
input_ids：文本对应的数字编码
attention_mask：标记哪些位置是实际文本（1），哪些是填充（0）
可能还有token_type_ids（针对某些模型的句子类型标记）
"""


"\n#['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask']\ninput_ids：文本对应的数字编码\nattention_mask：标记哪些位置是实际文本（1），哪些是填充（0）\n可能还有token_type_ids（针对某些模型的句子类型标记）\n"

In [ ]:
# 定义训练参数配置
training_args = TrainingArguments(
    output_dir="./model",  # 训练输出文件（模型、日志等）的保存目录
    eval_strategy="epoch",  # 评估策略：每完成一个训练轮次(epoch)后执行评估
    save_strategy="best",  # 模型保存策略："no"表示不自动保存模型checkpoint
    save_total_limit=3, #最多保存三个模型
    per_device_train_batch_size=32,  # 单设备上的训练批次大小（每个批次包含8个样本）
    per_device_eval_batch_size=32,  # 单设备上的评估批次大小（每个批次包含8个样本）
    num_train_epochs=6,  # 训练总轮次：整个数据集将被训练2遍
    learning_rate=2e-5,  # 学习率：BERT微调常用的学习率（2e-5即0.00002）
    logging_dir="./logs",  # 训练日志的保存目录（可用于TensorBoard可视化）
    logging_steps=50,  # 日志记录频率：每训练50个步骤(step)记录一次日志
    load_best_model_at_end=True,#训练结束后加载验证集效果最好的模型

    metric_for_best_model="accuracy",  # 定义"最佳模型"的评判指标（需与compute_metrics对应）
    greater_is_better=True,  # 指标是否越大越好（accuracy越大越好，loss则设为False）
    weight_decay=0.01,  # 权重衰减（防止过拟合，BERT微调常用）
    warmup_steps=500,  # 学习率预热步数（稳定训练初期的梯度）
    fp16=True,  # 若设备支持，开启混合精度训练（加速训练并节省显存）
    report_to="tensorboard",  # 日志报告到TensorBoard（默认也是，但显式指定更清晰）
)

In [11]:
#3 model
from sklearn.metrics import accuracy_score
model=BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

#计算准确率
def compute_metrics(eval_pred):
    logits,labels=eval_pred #数据集 结果和标签
    preds=torch.argmax(torch.tensor(logits),dim=-1) #激活函数
    acc=accuracy_score(labels,preds)
    return {"accuracy": acc}
    """
    logits：模型的原始输出（通常是未经过 softmax 激活的张量），形状一般为 (batch_size, num_classes)
    torch.argmax(..., dim=-1)：在最后一个维度（即类别维度）上取最大值的索引，也就是预测的类别
    """

#模型计算
trainer=Trainer( #模型，参数，数据集，准确率判断。 为什么没有优化函数呢？直接在model中定义好
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()
trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.668000,0.491450,0.817000
2,0.282500,0.275396,0.888750
3,0.223000,0.243163,0.912000
4,0.152300,0.417580,0.868250
5,0.068300,0.329303,0.908750
6,0.053500,0.340508,0.920750


{'eval_loss': 0.34050774574279785,
 'eval_accuracy': 0.92075,
 'eval_runtime': 17.1888,
 'eval_samples_per_second': 232.71,
 'eval_steps_per_second': 7.272,
 'epoch': 6.0}